# (1) New Dataset.

In [ ]:
import os

# ANA YOLU BELİRLE
base_dir = r"D:\data\manifest-1755254936487\LDCT-and-Projection-data"

print(f"Ana yol: {base_dir}")

# C ile başlayan hastaları bul ve ilk hastanın içeriğini listele
if os.path.exists(base_dir):
    hastalar = [d for d in os.listdir(base_dir) if d.startswith("C")]

    if len(hastalar) > 0:
        ilk_hasta = hastalar[0] # İlk C ile başlayan hasta
        hasta_yolu = os.path.join(base_dir, ilk_hasta)

        print(f"\n1. SEVİYE ({ilk_hasta} içi):")
        print(os.listdir(hasta_yolu))

        # İçerideki ilk alt klasöre bak
        icindekiler = os.listdir(hasta_yolu)
        if icindekiler:
            ilk_alt = icindekiler[0]
            alt_yol = os.path.join(hasta_yolu, ilk_alt)

            if os.path.isdir(alt_yol):
                print(f"\n2. SEVİYE ({ilk_alt} içi):")
                print(os.listdir(alt_yol))

                # Daha derine in
                derin_icindekiler = os.listdir(alt_yol)
                if derin_icindekiler:
                    derin_alt = derin_icindekiler[0]
                    derin_yol = os.path.join(alt_yol, derin_alt)
                    if os.path.isdir(derin_yol):
                         print(f"\n3. SEVİYE ({derin_alt} içi):")
                         print(os.listdir(derin_yol))
    else:
        print("C ile başlayan klasör bulunamadı!")
else:
    print("Ana yol yanlış!")

### This time we are going to use the DICOM format directly instead of converting it to PNG. Otherwise the images will be scientifically useless.

In [ ]:
import os
import pydicom
import cv2
import numpy as np

# === GÜNCELLENMİŞ DOSYA YOLLARI ===
# Senin verdiğin yeni doğru adres:
base_dir = r"D:\data\manifest-1755254936487\LDCT-and-Projection-data"

# Çıktı klasörünü de kontrol et istersen, buraya kaydedecek:
output_dir = r"D:\Projects\data\processed_data_npy"

os.makedirs(os.path.join(output_dir, "trainA"), exist_ok=True) # Low Dose
os.makedirs(os.path.join(output_dir, "trainB"), exist_ok=True) # High Dose

# === AYARLAR ===
HU_MIN = -1000
HU_MAX = 1000
IMG_SIZE = (256, 256)

def find_dose_folders(patient_path):
    low_p = None
    high_p = None

    # Tüm alt klasörleri gez
    for root, dirs, files in os.walk(patient_path):
        for d in dirs:
            d_lower = d.lower()

            # --- FİLTRE ---
            # 'proj' veya 'sino' içeren klasörleri atla
            if "proj" in d_lower or "sino" in d_lower:
                continue

            # Klasör eşleştirme
            if "low dose" in d_lower:
                low_p = os.path.join(root, d)
            elif "full dose" in d_lower or "high dose" in d_lower:
                high_p = os.path.join(root, d)

    return low_p, high_p


# C ve L ile başlayan hastaları alıyoruz
try:
    phantoms = [d for d in os.listdir(base_dir)
                if d.startswith(("C", "L"))
                and os.path.isdir(os.path.join(base_dir, d))]

    phantoms = sorted(phantoms)
    print(f"Bulunan Hastalar ({len(phantoms)} adet): {phantoms}")

except FileNotFoundError:
    print(f"HATA: '{base_dir}' yolu hala bulunamıyor. Lütfen D sürücüsünü kontrol et.")
    phantoms = []

print("\n--- İşlem Başlıyor ---\n")

for phantom in phantoms:
    patient_path = os.path.join(base_dir, phantom)

    # Low ve High Dose klasörlerini bul
    low_path, high_path = find_dose_folders(patient_path)

    # Klasörler bulundu mu kontrolü
    if not low_path or not high_path:
        print(f" Klasörler TAM bulunamadı (Atlanıyor -> Sadece projeksiyon olabilir): {phantom}")
        continue

    print(f" {phantom} işleniyor...")

    low_files = sorted(os.listdir(low_path))
    high_files = sorted(os.listdir(high_path))

    min_len = min(len(low_files), len(high_files))

    for i in range(min_len):
        low_f = low_files[i]
        high_f = high_files[i]

        try:
            # DICOM Oku
            low_dcm = pydicom.dcmread(os.path.join(low_path, low_f))
            high_dcm = pydicom.dcmread(os.path.join(high_path, high_f))

            # --- DÜZELTME BAŞLANGIÇ: HU Dönüşümü (Intercept & Slope Eklendi) ---

            # Low Dose için katsayılar
            intercept_low = low_dcm.RescaleIntercept if 'RescaleIntercept' in low_dcm else 0
            slope_low = low_dcm.RescaleSlope if 'RescaleSlope' in low_dcm else 1

            # High Dose için katsayılar
            intercept_high = high_dcm.RescaleIntercept if 'RescaleIntercept' in high_dcm else 0
            slope_high = high_dcm.RescaleSlope if 'RescaleSlope' in high_dcm else 1

            # Ham veriyi HU birimine çeviriyoruz
            low_img = low_dcm.pixel_array.astype(np.float32) * slope_low + intercept_low
            high_img = high_dcm.pixel_array.astype(np.float32) * slope_high + intercept_high

            # --- DÜZELTME BİTİŞ ---

            # 1. Windowing
            low_img = np.clip(low_img, HU_MIN, HU_MAX)
            high_img = np.clip(high_img, HU_MIN, HU_MAX)

            # 2. Resize
            low_img = cv2.resize(low_img, IMG_SIZE, interpolation=cv2.INTER_AREA)
            high_img = cv2.resize(high_img, IMG_SIZE, interpolation=cv2.INTER_AREA)

            # 3. Normalize (-1 ile 1 arası)
            low_img = (low_img - HU_MIN) / (HU_MAX - HU_MIN)
            high_img = (high_img - HU_MIN) / (HU_MAX - HU_MIN)

            low_img = (low_img * 2) - 1
            high_img = (high_img * 2) - 1

            # 4. Kaydet
            save_name = f"{phantom}_{i:04d}.npy"

            np.save(os.path.join(output_dir, "trainA", save_name), low_img.astype(np.float32))
            np.save(os.path.join(output_dir, "trainB", save_name), high_img.astype(np.float32))

        except Exception as e:
            print(f"   Hata ({phantom} - {i}): {e}")
            continue

    print(f"{phantom} bitti.")

print("\n TÜM İŞLEMLER BAŞARIYLA TAMAMLANDI.")

# (2) Model Building.

## This time the model will be a hybrid Pix2Pix + WGAN GP Model. Changes have been made accordingly.

### This part reads the data from the disk in .npy format.

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

# === AYARLAR ===
# Batch size'ı 4 yaptık, hem hızlı olur hem de RAM'i patlatmaz.
BATCH_SIZE = 4
IMG_WIDTH = 256
IMG_HEIGHT = 256

# === GÜNCELLENMİŞ SHUFFLE ÖZELLİKLİ DATASET SINIFI ===
class NPYDataset(tf.keras.utils.Sequence):
    def __init__(self, file_list_A, file_list_B, batch_size=1, shuffle=True):
        self.files_A = np.array(file_list_A) # Numpy array yapıyoruz
        self.files_B = np.array(file_list_B)
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(self.files_A)) # İndeks listesi

        print(f"Veri listesi hazırlandı... ({len(self.files_A)} adet dosya)")

        # Başlarken bir kere karıştıralım
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.files_A) / self.batch_size))

    def __getitem__(self, index):
        # O anki batch için indeksleri seç
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        batch_A = []
        batch_B = []

        for k in indexes:
            # Dosyayı yükle
            imgA = np.load(self.files_A[k])
            imgB = np.load(self.files_B[k])

            # Boyut Kontrolü (H, W) -> (H, W, 1)
            if imgA.ndim == 2:
                imgA = np.expand_dims(imgA, axis=-1)
            if imgB.ndim == 2:
                imgB = np.expand_dims(imgB, axis=-1)

            batch_A.append(imgA)
            batch_B.append(imgB)

        return np.array(batch_A), np.array(batch_B)

    def on_epoch_end(self):
        # Her epoch sonunda verileri karıştır (Shuffle)
        if self.shuffle:
            np.random.shuffle(self.indexes)

# === VERİYİ OKUMA VE AYIRMA ===
dataset_path = r"/content/drive/MyDrive/Tasarım Dersi/Projects/data/processed_data_npy"

# Tüm dosya yollarını diskten alıyoruz
all_files_A = sorted(glob.glob(os.path.join(dataset_path, 'trainA') + '/*.npy'))
all_files_B = sorted(glob.glob(os.path.join(dataset_path, 'trainB') + '/*.npy'))

print(f"Toplam bulunan dosya (trainA): {len(all_files_A)}")
print(f"Toplam bulunan dosya (trainB): {len(all_files_B)}")

# Dosya isimlerini al
filenames_A = {os.path.basename(f) for f in all_files_A}
filenames_B = {os.path.basename(f) for f in all_files_B}

# Eksik dosyaları bul
missing_in_B = filenames_A - filenames_B
missing_in_A = filenames_B - filenames_A

if missing_in_A:
    print(f"HATA: trainA içinde karşılığı olmayan dosya(lar) (trainB'de var): {missing_in_A}")
if missing_in_B:
    print(f"HATA: trainB içinde karşılığı olmayan dosya(lar) (trainA'da var): {missing_in_B}")

# Sadece eşleşen dosyaları al
common_filenames = sorted(list(filenames_A.intersection(filenames_B)))

# Yeni dosya listelerini oluştur
filtered_files_A = [os.path.join(dataset_path, 'trainA', f) for f in common_filenames]
filtered_files_B = [os.path.join(dataset_path, 'trainB', f) for f in common_filenames]

print(f"Eşleşen dosya sayısı: {len(filtered_files_A)}")

# %90 Train, %10 Validation ayırma
train_A, val_A, train_B, val_B = train_test_split(filtered_files_A, filtered_files_B, test_size=0.10, random_state=42)

print(f"Eğitim Seti: {len(train_A)} adet")
print(f"Test/Doğrulama Seti: {len(val_A)} adet")

# === DATASETLERİ OLUŞTURMA (Shuffle Eklendi) ===
# Train setini karıştırıyoruz (True), Validation sabit kalabilir (False)
train_dataset = NPYDataset(train_A, train_B, batch_size=BATCH_SIZE, shuffle=True)
val_dataset = NPYDataset(val_A, val_B, batch_size=BATCH_SIZE, shuffle=False)

Toplam bulunan dosya (trainA): 25473
Toplam bulunan dosya (trainB): 25472
HATA: trainB içinde karşılığı olmayan dosya(lar) (trainA'da var): {'C261_0000 (1).npy'}
Eşleşen dosya sayısı: 25472
Eğitim Seti: 22924 adet
Test/Doğrulama Seti: 2548 adet
Veri listesi hazırlandı... (22924 adet dosya)
Veri listesi hazırlandı... (2548 adet dosya)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Generator (U Net) and Discriminator (Critic). This time we are NOT using the Sigmoid function.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

IMG_WIDTH = 256
IMG_HEIGHT = 256
CHANNELS = 1

def downsample(filters, size, apply_batchnorm=True):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2D(filters, size, strides=2, padding='same',
                             kernel_initializer=initializer, use_bias=False))
    if apply_batchnorm:
        result.add(layers.BatchNormalization())
    result.add(layers.LeakyReLU())
    return result

def upsample(filters, size, apply_dropout=False):
    initializer = tf.random_normal_initializer(0., 0.02)
    result = keras.Sequential()
    result.add(layers.Conv2DTranspose(filters, size, strides=2, padding='same',
                                      kernel_initializer=initializer, use_bias=False))
    result.add(layers.BatchNormalization())
    if apply_dropout:
        result.add(layers.Dropout(0.5))
    result.add(layers.ReLU())
    return result

def build_generator():
    inputs = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS])

    # Encoder
    down_stack = [
        downsample(64, 4, apply_batchnorm=False), # (bs, 128, 128, 64)
        downsample(128, 4), # (bs, 64, 64, 128)
        downsample(256, 4), # (bs, 32, 32, 256)
        downsample(512, 4), # (bs, 16, 16, 512)
        downsample(512, 4), # (bs, 8, 8, 512)
        downsample(512, 4), # (bs, 4, 4, 512)
        downsample(512, 4), # (bs, 2, 2, 512)
        downsample(512, 4), # (bs, 1, 1, 512)
    ]

    # Decoder
    up_stack = [
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4, apply_dropout=True),
        upsample(512, 4),
        upsample(256, 4),
        upsample(128, 4),
        upsample(64, 4),
    ]

    initializer = tf.random_normal_initializer(0., 0.02)
    last = layers.Conv2DTranspose(CHANNELS, 4, strides=2, padding='same',
                                  kernel_initializer=initializer, activation='tanh')

    x = inputs
    skips = []
    for down in down_stack:
        x = down(x)
        skips.append(x)

    skips = reversed(skips[:-1])

    for up, skip in zip(up_stack, skips):
        x = up(x)
        x = layers.Concatenate()([x, skip])

    x = last(x)
    return keras.Model(inputs=inputs, outputs=x)

def build_discriminator():
    #   Sigmoid fonksiyonu yerine WGAN-GP kullanılmıştır.
    initializer = tf.random_normal_initializer(0., 0.02)

    inp = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS], name='input_image')
    tar = layers.Input(shape=[IMG_WIDTH, IMG_HEIGHT, CHANNELS], name='target_image')

    x = layers.Concatenate()([inp, tar]) # (bs, 256, 256, channels*2)

    down1 = downsample(64, 4, False)(x)
    down2 = downsample(128, 4)(down1)
    down3 = downsample(256, 4)(down2)

    # Zero Padding ve Conv
    zero_pad1 = layers.ZeroPadding2D()(down3)
    conv = layers.Conv2D(512, 4, strides=1, kernel_initializer=initializer, use_bias=False)(zero_pad1)
    batchnorm1 = layers.BatchNormalization()(conv)
    leaky_relu = layers.LeakyReLU()(batchnorm1)

    zero_pad2 = layers.ZeroPadding2D()(leaky_relu)

    last = layers.Conv2D(1, 4, strides=1, kernel_initializer=initializer)(zero_pad2)

    return keras.Model(inputs=[inp, tar], outputs=last)

generator = build_generator()
discriminator = build_discriminator()

### Hybrid Model Class (Pix2Pix + WGAN GP).

In [ ]:
class WGAN_GP_Pix2Pix(keras.Model):
    def __init__(self, generator, discriminator, lambda_gp=10.0, lambda_l1=100.0):
        super(WGAN_GP_Pix2Pix, self).__init__()
        self.generator = generator
        self.discriminator = discriminator
        self.lambda_gp = lambda_gp # Gradient Penalty ağırlığı
        self.lambda_l1 = lambda_l1 # L1 (Pix2Pix) ağırlığı

    def compile(self, d_optimizer, g_optimizer):
        super(WGAN_GP_Pix2Pix, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = self.wasserstein_loss
        self.g_loss_fn = self.wasserstein_loss
        self.l1_loss_fn = tf.keras.losses.MeanAbsoluteError()

    def wasserstein_loss(self, y_true, y_pred):
        return tf.reduce_mean(y_true * y_pred)

    def gradient_penalty(self, batch_size, real_images, fake_images, input_images):
        """ GP Hesaplama: Real ve Fake arası interpolasyon """
        alpha = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            # Discriminator'a hem input(LD) hem interpolasyon verilir
            pred = self.discriminator([input_images, interpolated], training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp

    def call(self, inputs, training=False):
        if isinstance(inputs, (list, tuple)):
            inputs = inputs[0]
        return self.generator(inputs, training=training)

    def train_step(self, data):
        # Data Loader'dan gelen veri: (input_image, target_image)
        input_image, target_image = data
        batch_size = tf.shape(input_image)[0]

        # --- DISCRIMINATOR EĞİTİMİ ---
        with tf.GradientTape() as tape:
            fake_image = self.generator(input_image, training=True)

            fake_pred = self.discriminator([input_image, fake_image], training=True)
            real_pred = self.discriminator([input_image, target_image], training=True)

            # Wasserstein Loss: D(fake) - D(real)

            # Not: Real için -1, Fake için 1 gibi davranılır, formül minimize etmek üzerine kuruludur.

            d_cost = tf.reduce_mean(fake_pred) - tf.reduce_mean(real_pred)

            # Gradient Penalty
            gp = self.gradient_penalty(batch_size, target_image, fake_image, input_image)

            # Toplam D Loss
            d_loss = d_cost + (gp * self.lambda_gp)

        d_grad = tape.gradient(d_loss, self.discriminator.trainable_variables)
        self.d_optimizer.apply_gradients(zip(d_grad, self.discriminator.trainable_variables))

        # --- GENERATOR EĞİTİMİ ---
        with tf.GradientTape() as tape:
            fake_image = self.generator(input_image, training=True)
            fake_pred = self.discriminator([input_image, fake_image], training=True)

            # G Loss (Wasserstein Kısmı)
            g_wgan_loss = -tf.reduce_mean(fake_pred)

            # G Loss (L1 Kısmı): Orijinal Pix2Pix yapısı (Görüntü benzerliği)
            g_l1_loss = self.l1_loss_fn(target_image, fake_image) * self.lambda_l1

            g_loss = g_wgan_loss + g_l1_loss

        g_grad = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_grad, self.generator.trainable_variables))

        return {"d_loss": d_loss, "g_loss": g_loss, "g_l1": g_l1_loss}

# (3) Compiling and Training the Model.

In [ ]:
# === GPU KONTROL ===

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [ ]:
import os
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# === AYARLAR ===
LEARNING_RATE_G = 2e-4
LEARNING_RATE_D = 2e-4
# Batch size'ı yukarıda ayarlamıştık zaten, burada tekrar tanımlamaya gerek yok

# === 1. ADIM: KLASÖRLER ===
project_root = r"D:\Projects"
results_dir = os.path.join(project_root, "results")
checkpoint_dir = os.path.join(project_root, "model_checkpoints")

os.makedirs(results_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)

# === 2. ADIM: MODELİ DERLE ===
hybrid_gan = WGAN_GP_Pix2Pix(generator=generator, discriminator=discriminator)

hybrid_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_D, beta_1=0.5, beta_2=0.9),
    g_optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE_G, beta_1=0.5, beta_2=0.9)
)

# === 3. ADIM: SADE MONİTÖR (SENİN KODUN + KAYIT ÖZELLİĞİ) ===
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, val_dataset, num_img=3):
        self.val_dataset = val_dataset
        self.num_img = num_img

    def on_epoch_end(self, epoch, logs=None):
        # --- A. GÖRSEL KAYIT (HER EPOCH) ---
        print(f"\nEpoch {epoch+1} bitti. Görseller işleniyor...")

        try:
            # Rastgele veri çek
            idx = np.random.randint(0, len(self.val_dataset))
            inp, tar = self.val_dataset[idx]
            prediction = self.model.generator(inp, training=False)

            title = ['Input (LD)', 'Generated (AI)', 'Target (HD)']
            display_count = min(self.num_img, inp.shape[0])

            for i in range(display_count):
                img_list = [inp[i], prediction[i], tar[i]]

                plt.figure(figsize=(12, 4))
                for j in range(3):
                    plt.subplot(1, 3, j+1)
                    plt.title(title[j])

                    # --- TEK DEĞİŞİKLİK BURASI (BEYAZLIK GİTMESİ İÇİN) ---
                    # Veriyi alıp en küçük ve en büyüğe göre 0-1 arasına çekiyoruz.
                    # Bu sayede veri bozuk bile olsa ekranda gri gözükür.
                    img_data = img_list[j][:, :, 0]
                    _min, _max = np.min(img_data), np.max(img_data)

                    if _max - _min > 0:
                        show_img = (img_data - _min) / (_max - _min)
                    else:
                        show_img = img_data

                    plt.imshow(show_img, cmap='gray')
                    plt.axis('off')

                filename = f"epoch_{epoch+1}_{i}.png"
                save_path = os.path.join(results_dir, filename)
                plt.savefig(save_path)
                plt.close() # Sadece figürü kapatır, RAM temizlemez. Güvenli.

            # --- B. MODEL KAYIT (HER 5 EPOCH) ---
            if (epoch+1) % 5 == 0:
                model_name = f"G_epoch_{epoch+1}.h5"
                ckpt_path = os.path.join(checkpoint_dir, model_name)
                self.model.generator.save_weights(ckpt_path)
                print(f"✅ Model Kaydedildi: {ckpt_path}")

        except Exception as e:
            print(f"Kayıt hatası: {e}")

# === 4. ADIM: BAŞLAT ===
"""print("Eğitim başlıyor... (Eski performans, yeni kayıt sistemi)")

hybrid_gan.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=50,
    callbacks=[GANMonitor(val_dataset)]
)"""

'print("Eğitim başlıyor... (Eski performans, yeni kayıt sistemi)")\n\nhybrid_gan.fit(\n    train_dataset,\n    validation_data=val_dataset,\n    epochs=50,\n    callbacks=[GANMonitor(val_dataset)]\n)'

# Evaluating the Generator Model with PSNR and SSIM

In [ ]:
import os
import numpy as np
import tensorflow as tf

# Assuming the generator and discriminator models are defined as in previous cells
# Re-define build_generator and build_discriminator functions if they are not in scope.
# For this task, we only need the generator.

# --- 1. Load the latest Generator Model ---
# Make sure the `build_generator` function from the previous cells is run before this cell.
# You might need to adjust the epoch number based on your saved model.

checkpoint_dir = "/content/drive/MyDrive/Tasarım Dersi/Projects/model_checkpoints"

# Find the latest saved generator model
# Assuming your model names are like G_epoch_XX.h5
list_of_files = glob.glob(os.path.join(checkpoint_dir, 'G_epoch_*.h5'))
if not list_of_files:
    print("HATA: Kaydedilmiş bir Generator modeli bulunamadı. Lütfen önce modeli eğittiğinizden ve kaydettiğinizden emin olun.")
    # Exit or handle error appropriately
else:
    latest_file = max(list_of_files, key=os.path.getctime)
    print(f"En son kaydedilen model: {latest_file}")

    # Re-build the generator architecture (it needs to be the same as trained)
    # If 'generator' object from `yuyDVrxlJ6BC` is still in memory, you can skip this line
    # from build_generator import build_generator # If you saved build_generator in a separate file
    if 'generator' not in locals():
        generator = build_generator()

    # Load weights into the generator
    try:
        generator.load_weights(latest_file)
        print("Jeneratör modeli başarıyla yüklendi.")
    except Exception as e:
        print(f"Model yüklenirken bir hata oluştu: {e}")
        generator = None # Set to None to prevent further errors if loading fails


if generator:
    # --- 2. Calculate PSNR and SSIM on the validation dataset ---
    psnr_scores = []
    ssim_scores = []

    print("Doğrulama seti üzerinde PSNR ve SSIM hesaplanıyor...")

    for batch_idx, (input_image, target_image) in enumerate(val_dataset):
        # Generate prediction
        predicted_image = generator(input_image, training=False)

        # Ensure images are in the correct format (float32) and range for PSNR/SSIM
        # PSNR and SSIM typically expect values in [0, 1] or [-1, 1].
        # Our normalization was to [-1, 1], so we'll use that range for max_val.

        # Calculate PSNR
        batch_psnr = tf.image.psnr(target_image, predicted_image, max_val=2.0) # Max value is 1 - (-1) = 2
        psnr_scores.extend(batch_psnr.numpy().tolist())

        # Calculate SSIM
        # SSIM requires a channel dimension if it's not already there (which it should be for our images)
        batch_ssim = tf.image.ssim(target_image, predicted_image, max_val=2.0) # Max value is 1 - (-1) = 2
        ssim_scores.extend(batch_ssim.numpy().tolist())

        if batch_idx % 100 == 0: # Print progress
            print(f"  İşlenen batch: {batch_idx}/{len(val_dataset)}")

    # --- 3. Report Average Scores ---
    avg_psnr = np.mean(psnr_scores)
    avg_ssim = np.mean(ssim_scores)

    print(f"\nOrtalama PSNR: {avg_psnr:.4f}")
    print(f"Ortalama SSIM: {avg_ssim:.4f}")
else:
    print("Jeneratör modeli yüklenemediği için değerlendirme yapılamadı.")

En son kaydedilen model: /content/drive/MyDrive/Tasarım Dersi/Projects/model_checkpoints/G_epoch_50.h5
Model yüklenirken bir hata oluştu: 'NoneType' object has no attribute 'load_weights'
Jeneratör modeli yüklenemediği için değerlendirme yapılamadı.


### Evaluate 2 (Alternate)

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
from tqdm.notebook import tqdm

# === 1. MODELİ HAZIRLA ===
if 'checkpoint_dir' not in locals():
    # Colab yolu
    checkpoint_dir = "/content/drive/MyDrive/Tasarım Dersi/Projects/model_checkpoints"

# Modeli bul ve yükle
list_of_files = glob.glob(os.path.join(checkpoint_dir, 'G_epoch_*.h5'))
if not list_of_files:
    raise ValueError("Model dosyası bulunamadı!")

latest_file = max(list_of_files, key=os.path.getctime)
print(f"Yüklenen Model: {os.path.basename(latest_file)}")

generator = build_generator()
generator.load_weights(latest_file)
print("✅ Model hazır.")

# === 2. VERİYİ RAM'E ÇEK (Hızlı Yöntem) ===
val_files_A = val_dataset.files_A
val_files_B = val_dataset.files_B

print(f"\nVeriler RAM'e yükleniyor (Yaklaşık 40-50 sn sürer)...")
all_inputs = []
all_targets = []

for fA, fB in tqdm(zip(val_files_A, val_files_B), total=len(val_files_A)):
    try:
        imgA = np.load(fA)
        imgB = np.load(fB)

        # Boyut düzeltme
        if imgA.ndim == 2: imgA = np.expand_dims(imgA, axis=-1)
        if imgB.ndim == 2: imgB = np.expand_dims(imgB, axis=-1)

        all_inputs.append(imgA)
        all_targets.append(imgB)
    except:
        continue

X_val = np.array(all_inputs)
Y_val = np.array(all_targets)
print(f"✅ Veri Hazır. Toplam Görüntü: {len(X_val)}")

# === 3. GÜVENLİ HESAPLAMA (Safe Mode) ===
# CPU'yu yormamak için: Tahmin et -> Hesapla -> Unut yöntemini kullanıyoruz.
print("\nAnaliz Başlıyor (CPU Modu - Adım Adım)...")

psnr_scores = []
ssim_scores = []

# İşlemciyi kilitlememek için 50'şerli paketler halinde yapıyoruz
batch_size = 50
total_samples = len(X_val)

for i in tqdm(range(0, total_samples, batch_size)):
    # 1. Küçük bir parça al
    batch_input = X_val[i : i + batch_size]
    batch_target = Y_val[i : i + batch_size]

    # 2. Sadece bu parça için tahmin yap (Training=False)
    batch_pred = generator(batch_input, training=False)

    # 3. Sadece bu parça için PSNR/SSIM hesapla
    # (Veriler -1 ile 1 arasında olduğu için max_val=2.0)
    batch_psnr = tf.image.psnr(batch_target, batch_pred, max_val=2.0).numpy()
    batch_ssim = tf.image.ssim(batch_target, batch_pred, max_val=2.0).numpy()

    # 4. Sonuçları listeye ekle
    psnr_scores.extend(batch_psnr)
    ssim_scores.extend(batch_ssim)

# === 4. SONUÇLARI YAZDIR ===
avg_psnr = np.mean(psnr_scores)
std_psnr = np.std(psnr_scores)
avg_ssim = np.mean(ssim_scores)
std_ssim = np.std(ssim_scores)

print("\n" + "="*40)
print(f"📊 FİNAL SONUÇLAR")
print("="*40)
print(f"Ortalama PSNR: {avg_psnr:.4f} dB (±{std_psnr:.4f})")
print(f"Ortalama SSIM: {avg_ssim:.4f} (±{std_ssim:.4f})")
print("="*40)

Yüklenen Model: G_epoch_50.h5
✅ Model hazır.

Veriler RAM'e yükleniyor (Yaklaşık 40-50 sn sürer)...


  0%|          | 0/2548 [00:00<?, ?it/s]

✅ Veri Hazır. Toplam Görüntü: 2548

Analiz Başlıyor (CPU Modu - Adım Adım)...


  0%|          | 0/51 [00:00<?, ?it/s]


📊 FİNAL SONUÇLAR
Ortalama PSNR: 41.7306 dB (±5.8375)
Ortalama SSIM: 0.9411 (±0.0452)


# Evaluating the Model with New Dataset

In [ ]:
!pip install pydicom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 62.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pydicom
from pathlib import Path

# ==================== AYARLAR ====================
phantomx_base_dir = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/phantomx_abdomen_pelvis_dataset"
output_dir = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Output"

phantoms = ["D55-01", "D55-02"]

print("=" * 80)
print("PhantomX DICOM → NPY Dönüştürücü (3 Seviyeli Yapı)")
print("=" * 80)
print(f"📂 Ana dizin: {phantomx_base_dir}")
print(f"📂 Çıktı dizini: {output_dir}")

# Dizin kontrolü
if not os.path.exists(phantomx_base_dir):
    print(f"\n❌ HATA: Ana dizin bulunamadı!")
    print(f"   Kontrol edin: {phantomx_base_dir}")
    exit()

# ==================== KLASÖR YAPISINI ANLA ====================

def klasor_yapisini_incele(base_dir, phantoms):
    """3 seviyeli klasör yapısını inceler"""

    print("\n" + "="*80)
    print("📋 KLASÖR YAPISI ANALİZİ")
    print("="*80)

    klasor_bilgileri = {}

    for phantom in phantoms:
        phantom_yolu = os.path.join(base_dir, phantom)

        if not os.path.exists(phantom_yolu):
            print(f"\n❌ {phantom} klasörü bulunamadı!")
            continue

        print(f"\n{'─'*80}")
        print(f"🔬 Phantom: {phantom}")
        print(f"{'─'*80}")

        # 1. Seviye: Doz klasörleri (40, 300)
        doz_klasorleri = {}

        for doz_adi in sorted(os.listdir(phantom_yolu)):
            doz_yolu = os.path.join(phantom_yolu, doz_adi)

            if not os.path.isdir(doz_yolu):
                continue

            print(f"\n  📁 Doz Klasörü: {doz_adi}")

            # Doz bilgisini belirle
            if doz_adi == "40":
                doz_tipi = "1.4 CTDIvol (Düşük Doz)"
            elif doz_adi == "300":
                doz_tipi = "10.5 CTDIvol (Yüksek Doz)"
            else:
                doz_tipi = "Bilinmeyen"

            print(f"      → {doz_tipi}")

            # 2. Seviye: Rekonstrüksiyon metodu klasörleri
            rekon_klasorleri = []

            for rekon_adi in sorted(os.listdir(doz_yolu)):
                rekon_yolu = os.path.join(doz_yolu, rekon_adi)

                if not os.path.isdir(rekon_yolu):
                    continue

                # DICOM sayısını say
                dicom_sayisi = len([f for f in os.listdir(rekon_yolu) if f.endswith('.dcm')])

                # Rekonstrüksiyon tipini belirle
                if "FBP" in rekon_adi.upper():
                    rekon_tip = "FBP (Filtered Back Projection)"
                elif "AIDR" in rekon_adi.upper():
                    rekon_tip = "AIDR3D (Iterative Reconstruction)"
                elif "AICE" in rekon_adi.upper() or "ICE" in rekon_adi.upper():
                    rekon_tip = "AiCE (AI-based Reconstruction)"
                else:
                    rekon_tip = "Bilinmeyen"

                rekon_klasorleri.append({
                    'ad': rekon_adi,
                    'yol': rekon_yolu,
                    'tip': rekon_tip,
                    'dicom_sayisi': dicom_sayisi
                })

                print(f"      └── {rekon_tip}")
                print(f"          Klasör: {rekon_adi}")
                print(f"          DICOM: {dicom_sayisi} dosya")

            doz_klasorleri[doz_adi] = {
                'yol': doz_yolu,
                'tip': doz_tipi,
                'rekonstruksiyonlar': rekon_klasorleri
            }

        klasor_bilgileri[phantom] = doz_klasorleri

    return klasor_bilgileri

# Klasör yapısını incele
klasor_bilgileri = klasor_yapisini_incele(phantomx_base_dir, phantoms)

if not klasor_bilgileri:
    print("\n❌ Hiçbir phantom klasörü bulunamadı!")
    exit()

# ==================== KULLANICIDAN SEÇİM AL ====================

print("\n" + "="*80)
print("🎯 REKONSTRÜKSİYON METODU SEÇİMİ")
print("="*80)
print("\n⚠️  ÖNERİ: FBP metodu seçin (model eğitiminde kullanıldıysa)")
print("   • FBP: Standart, ham gürültü içerir (en yaygın)")
print("   • AIDR3D: Iterative, kısmen gürültü azaltılmış")
print("   • AiCE: AI-based, yoğun gürültü azaltma yapılmış")

# Hangi rekonstrüksiyon metodunu kullanacağız?
print("\nHangi rekonstrüksiyon metodunu kullanmak istersiniz?")
print("  [1] FBP (Önerilen)")
print("  [2] AIDR3D")
print("  [3] AiCE")

rekon_secim = input("\nSeçiminiz (1/2/3): ").strip()

rekon_map = {
    "1": "FBP",
    "2": "AIDR",
    "3": "AICE"
}

if rekon_secim not in rekon_map:
    print("❌ Geçersiz seçim! Varsayılan olarak FBP kullanılacak.")
    rekon_secim = "1"

secilen_rekon = rekon_map[rekon_secim]
print(f"\n✅ Seçilen metod: {secilen_rekon}")

# ==================== FONKSİYONLAR ====================

def rekonstruksiyon_klasoru_bul(rekon_klasorleri, aranan_tip):
    """Belirtilen rekonstrüksiyon tipine ait klasörü bulur"""
    for klasor in rekon_klasorleri:
        if aranan_tip.upper() in klasor['ad'].upper():
            return klasor
    return None

def dicom_klasoru_oku(klasor_yolu, klasor_adi):
    """DICOM dosyalarını okur ve 3D numpy array olarak döndürür"""
    print(f"      📂 {klasor_adi}")

    dicom_dosyalari = []

    for dosya in os.listdir(klasor_yolu):
        if dosya.endswith('.dcm'):
            dosya_yolu = os.path.join(klasor_yolu, dosya)
            dicom_dosyalari.append(dosya_yolu)

    if not dicom_dosyalari:
        print(f"      ❌ DICOM dosyası bulunamadı!")
        return None

    print(f"      ✓ {len(dicom_dosyalari)} DICOM dosyası bulundu")

    # DICOM dosyalarını oku ve sırala
    dicom_veriler = []
    hata_sayisi = 0

    for dosya in sorted(dicom_dosyalari):
       try:
            ds = pydicom.dcmread(dosya)

            # Pixel array'i al
            pixel_array = ds.pixel_array.astype(np.float32)

            # --- DÜZELTME BAŞLANGIÇ: HU Dönüşümü ---
            intercept = ds.RescaleIntercept if 'RescaleIntercept' in ds else 0
            slope = ds.RescaleSlope if 'RescaleSlope' in ds else 1

            # Ham veriyi HU birimine çevir
            pixel_array = pixel_array * slope + intercept
            # --- DÜZELTME BİTİŞ ---

            # Sonra direkt clip ve normalize işlemlerine geç
            pixel_array = np.clip(pixel_array, -1000, 1000)

            # Z koordinatı veya InstanceNumber kullan
            if hasattr(ds, 'ImagePositionPatient'):
                slice_location = float(ds.ImagePositionPatient[2])
            elif hasattr(ds, 'SliceLocation'):
                slice_location = float(ds.SliceLocation)
            elif hasattr(ds, 'InstanceNumber'):
                slice_location = float(ds.InstanceNumber)
            else:
                slice_location = len(dicom_veriler)

            dicom_veriler.append((slice_location, pixel_array))

        except Exception as e:
            hata_sayisi += 1
            if hata_sayisi <= 3:
                print(f"      ⚠ Hata ({os.path.basename(dosya)}): {e}")

    if hata_sayisi > 3:
        print(f"      ⚠ Toplam {hata_sayisi} dosya okunamadı")

    if not dicom_veriler:
        print(f"      ❌ Hiçbir DICOM dosyası okunamadı!")
        return None

    # Z koordinatına göre sırala
    dicom_veriler.sort(key=lambda x: x[0])

    # 3D array oluştur
    volume_3d = np.stack([slice_data for _, slice_data in dicom_veriler])

    print(f"      ✓ Volume: {volume_3d.shape}")
    print(f"      ✓ HU aralığı: [{volume_3d.min():.1f}, {volume_3d.max():.1f}]")

    return volume_3d

def normalizasyon(volume, min_hu=-1000, max_hu=1000):
    """HU değerlerini [-1, 1] aralığına normalize eder"""
    volume = np.clip(volume, min_hu, max_hu)
    volume = (volume - min_hu) / (max_hu - min_hu)  # [0, 1]
    volume = volume * 2 - 1  # [-1, 1]
    return volume.astype(np.float32)

# ==================== DÖNÜŞTÜRMEYİ BAŞLAT ====================

print("\n" + "="*80)
print("🔄 DÖNÜŞTÜRME İŞLEMİ")
print("="*80)

os.makedirs(output_dir, exist_ok=True)

basarili_donusumler = []

for phantom_name, doz_klasorleri in klasor_bilgileri.items():
    print(f"\n{'='*80}")
    print(f"🔬 Phantom: {phantom_name}")
    print(f"{'='*80}")

    # 40 = düşük doz (INPUT)
    # 300 = yüksek doz (TARGET)

    if "40" not in doz_klasorleri or "300" not in doz_klasorleri:
        print(f"  ❌ Gerekli doz klasörleri bulunamadı (40 veya 300 eksik)")
        continue

    # INPUT: 40 klasöründen seçilen rekonstrüksiyon
    input_rekonlar = doz_klasorleri["40"]["rekonstruksiyonlar"]
    input_rekon = rekonstruksiyon_klasoru_bul(input_rekonlar, secilen_rekon)

    # TARGET: 300 klasöründen seçilen rekonstrüksiyon
    target_rekonlar = doz_klasorleri["300"]["rekonstruksiyonlar"]
    target_rekon = rekonstruksiyon_klasoru_bul(target_rekonlar, secilen_rekon)

    if not input_rekon or not target_rekon:
        print(f"  ❌ {secilen_rekon} rekonstrüksiyonu bulunamadı!")
        continue

    print(f"\n  📥 INPUT (1.4 CTDIvol):")
    print(f"     {input_rekon['ad']}")
    print(f"  📤 TARGET (10.5 CTDIvol):")
    print(f"     {target_rekon['ad']}")

    try:
        # DICOM'ları oku
        print(f"\n  📁 DICOM dosyaları okunuyor...")

        print(f"\n    INPUT:")
        input_volume = dicom_klasoru_oku(input_rekon['yol'], input_rekon['ad'])

        print(f"\n    TARGET:")
        target_volume = dicom_klasoru_oku(target_rekon['yol'], target_rekon['ad'])

        if input_volume is None or target_volume is None:
            print(f"\n  ❌ DICOM okuma başarısız!")
            continue

        # Shape kontrolü
        if input_volume.shape != target_volume.shape:
            print(f"\n  ⚠ UYARI: Boyutlar farklı!")
            print(f"    Input: {input_volume.shape}")
            print(f"    Target: {target_volume.shape}")

            min_slices = min(input_volume.shape[0], target_volume.shape[0])
            min_height = min(input_volume.shape[1], target_volume.shape[1])
            min_width = min(input_volume.shape[2], target_volume.shape[2])

            input_volume = input_volume[:min_slices, :min_height, :min_width]
            target_volume = target_volume[:min_slices, :min_height, :min_width]

            print(f"    → Uyumlu hale getirildi: {input_volume.shape}")

        # Normalizasyon
        print(f"\n  🔄 Normalizasyon: [-1000, 1000] HU → [-1, 1]")

        input_normalized = normalizasyon(input_volume)
        target_normalized = normalizasyon(target_volume)

        print(f"    ✓ Input: [{input_normalized.min():.3f}, {input_normalized.max():.3f}]")
        print(f"    ✓ Target: [{target_normalized.min():.3f}, {target_normalized.max():.3f}]")

        # NPY olarak kaydet
        print(f"\n  💾 Dosyalar kaydediliyor...")
        phantom_output = os.path.join(output_dir, phantom_name)
        os.makedirs(phantom_output, exist_ok=True)

        input_save_path = os.path.join(phantom_output, "input_ldct.npy")
        target_save_path = os.path.join(phantom_output, "target_ndct.npy")

        np.save(input_save_path, input_normalized)
        np.save(target_save_path, target_normalized)

        print(f"    ✓ {input_save_path}")
        print(f"    ✓ {target_save_path}")

        # İstatistikler
        print(f"\n  📊 İstatistikler:")
        print(f"     Slice: {input_normalized.shape[0]}")
        print(f"     Boyut: {input_normalized.shape[1]} x {input_normalized.shape[2]}")
        print(f"     Dosya: {input_normalized.nbytes / (1024**2):.2f} MB (her biri)")

        basarili_donusumler.append(phantom_name)
        print(f"\n  ✅ Başarılı!")

    except Exception as e:
        print(f"\n  ❌ HATA: {e}")
        import traceback
        traceback.print_exc()

# ==================== ÖZET ====================

print("\n" + "="*80)
if basarili_donusumler:
    print("✅ DÖNÜŞTÜRME TAMAMLANDI!")
    print("="*80)
    print(f"\n📁 Çıktı: {output_dir}\n")
    for phantom in basarili_donusumler:
        print(f"  {phantom}/")
        print(f"    ├── input_ldct.npy  (1.4 CTDIvol - Düşük Doz)")
        print(f"    └── target_ndct.npy (10.5 CTDIvol - Yüksek Doz)")
    print(f"\n🚀 Test kodunu çalıştırabilirsiniz!")
else:
    print("❌ HİÇBİR PHANTOM DÖNÜŞTÜRÜLEMEDI!")
    print("="*80)
print()

PhantomX DICOM → NPY Dönüştürücü (3 Seviyeli Yapı)
📂 Ana dizin: /content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/phantomx_abdomen_pelvis_dataset
📂 Çıktı dizini: /content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Output

📋 KLASÖR YAPISI ANALİZİ

────────────────────────────────────────────────────────────────────────────────
🔬 Phantom: D55-01
────────────────────────────────────────────────────────────────────────────────

  📁 Doz Klasörü: 300
      → 10.5 CTDIvol (Yüksek Doz)
      └── AIDR3D (Iterative Reconstruction)
          Klasör: AIDR3D_FC08_300_171515.916
          DICOM: 376 dosya
      └── AiCE (AI-based Reconstruction)
          Klasör: AiCE_BODY-SHARP_300_172938.900
          DICOM: 376 dosya
      └── FBP (Filtered Back Projection)
          Klasör: FBP_FC08_300_171515.916
          DICOM: 376 dosya

  📁 Doz Klasörü: 40
      → 1.4 CTDIvol (Düşük Doz)
      └── AIDR3D (Iterative Reconstruction)
          Klasör: AIDR3D_FC08_40_152622.890
          D

In [ ]:
import os
import glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from tqdm import tqdm

# ==================== AYARLAR ====================
phantomx_processed_dir = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Output"
checkpoint_dir = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Projects/model_checkpoints"
output_dir = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Results2"

phantoms = ["D55-01", "D55-02"]

print("=" * 80)
print("PhantomX Model Test ve Değerlendirme")
print("=" * 80)
print(f"📂 Processed data: {phantomx_processed_dir}")
print(f"🧠 Model checkpoints: {checkpoint_dir}")
print(f"📊 Results output: {output_dir}")
print("=" * 80)

# ==================== FONKSİYONLAR ====================

def load_latest_generator(checkpoint_dir, generator_model):
    """En yüksek epoch numaralı generator modelini yükler"""
    list_of_files = glob.glob(os.path.join(checkpoint_dir, 'G_epoch_*.h5'))

    if not list_of_files:
        print("❌ HATA: Kaydedilmiş generator modeli bulunamadı!")
        print(f"   Kontrol edilen dizin: {checkpoint_dir}")
        print(f"   Aranan format: G_epoch_*.h5")

        # Klasördeki tüm dosyaları göster
        if os.path.exists(checkpoint_dir):
            all_files = os.listdir(checkpoint_dir)
            if all_files:
                print(f"\n   Klasördeki dosyalar:")
                for f in sorted(all_files)[:10]:  # İlk 10 dosyayı göster
                    print(f"   - {f}")
        return None

    # Epoch numarasına göre sırala ve en büyüğünü al
    def get_epoch_number(filepath):
        """Dosya adından epoch numarasını çıkar"""
        basename = os.path.basename(filepath)
        # G_epoch_50.h5 -> 50
        try:
            epoch_str = basename.split('_')[-1].replace('.h5', '')
            return int(epoch_str)
        except:
            return 0

    latest_file = max(list_of_files, key=get_epoch_number)
    epoch_num = get_epoch_number(latest_file)

    print(f"\n📦 Model bulundu ve yükleniyor...")
    print(f"   Dosya: {os.path.basename(latest_file)}")
    print(f"   Epoch: {epoch_num}")
    print(f"   Toplam {len(list_of_files)} checkpoint bulundu")

    try:
        generator_model.load_weights(latest_file)
        print("✅ Model başarıyla yüklendi!")
        return generator_model
    except Exception as e:
        print(f"❌ Model yükleme hatası: {e}")
        print(f"\n💡 Model mimarisi ile checkpoint uyumsuz olabilir.")
        print(f"   Lütfen generator modelinin doğru tanımlandığından emin olun.")
        return None

def calculate_metrics_3d(target, predicted):
    """3D volume için PSNR ve SSIM hesaplar"""
    psnr_scores = []
    ssim_scores = []

    # Her slice için ayrı ayrı hesapla
    for i in range(target.shape[0]):
        target_slice = target[i]
        pred_slice = predicted[i]

        # PSNR hesaplama (data_range=2.0 çünkü [-1, 1] aralığında)
        psnr_val = psnr(target_slice, pred_slice, data_range=2.0)
        psnr_scores.append(psnr_val)

        # SSIM hesaplama
        ssim_val = ssim(target_slice, pred_slice, data_range=2.0)
        ssim_scores.append(ssim_val)

    return np.array(psnr_scores), np.array(ssim_scores)

def visualize_results(input_vol, target_vol, predicted_vol, phantom_name, output_dir, num_slices=5):
    """Sonuçları görselleştirir ve kaydeder"""
    total_slices = input_vol.shape[0]
    slice_indices = np.linspace(0, total_slices-1, num_slices, dtype=int)

    fig, axes = plt.subplots(num_slices, 4, figsize=(16, 4*num_slices))

    for idx, slice_num in enumerate(slice_indices):
        # Input (LDCT)
        axes[idx, 0].imshow(input_vol[slice_num], cmap='gray', vmin=-1, vmax=1)
        axes[idx, 0].set_title(f'Input (LDCT)\nSlice {slice_num}/{total_slices}', fontsize=10)
        axes[idx, 0].axis('off')

        # Target (NDCT)
        axes[idx, 1].imshow(target_vol[slice_num], cmap='gray', vmin=-1, vmax=1)
        axes[idx, 1].set_title(f'Target (NDCT)\nSlice {slice_num}/{total_slices}', fontsize=10)
        axes[idx, 1].axis('off')

        # Predicted
        axes[idx, 2].imshow(predicted_vol[slice_num], cmap='gray', vmin=-1, vmax=1)
        axes[idx, 2].set_title(f'Predicted\nSlice {slice_num}/{total_slices}', fontsize=10)
        axes[idx, 2].axis('off')

        # Difference map
        diff = np.abs(target_vol[slice_num] - predicted_vol[slice_num])
        im = axes[idx, 3].imshow(diff, cmap='hot', vmin=0, vmax=0.5)
        axes[idx, 3].set_title(f'Absolute Error\nSlice {slice_num}/{total_slices}', fontsize=10)
        axes[idx, 3].axis('off')

        # Colorbar for error map
        plt.colorbar(im, ax=axes[idx, 3], fraction=0.046, pad=0.04)

    plt.tight_layout()

    # Kaydet
    save_path = os.path.join(output_dir, f"{phantom_name}_comparison.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"  💾 Görselleştirme kaydedildi: {os.path.basename(save_path)}")
    plt.close()

def plot_metrics_distribution(psnr_scores, ssim_scores, phantom_name, output_dir):
    """Metrik dağılımlarını çizer"""
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # PSNR histogram
    axes[0].hist(psnr_scores, bins=30, color='steelblue', edgecolor='black', alpha=0.7)
    axes[0].axvline(np.mean(psnr_scores), color='red', linestyle='--', linewidth=2,
                    label=f'Ortalama: {np.mean(psnr_scores):.2f} dB')
    axes[0].set_xlabel('PSNR (dB)', fontsize=12)
    axes[0].set_ylabel('Slice Sayısı', fontsize=12)
    axes[0].set_title(f'{phantom_name} - PSNR Dağılımı', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)

    # SSIM histogram
    axes[1].hist(ssim_scores, bins=30, color='forestgreen', edgecolor='black', alpha=0.7)
    axes[1].axvline(np.mean(ssim_scores), color='red', linestyle='--', linewidth=2,
                    label=f'Ortalama: {np.mean(ssim_scores):.4f}')
    axes[1].set_xlabel('SSIM', fontsize=12)
    axes[1].set_ylabel('Slice Sayısı', fontsize=12)
    axes[1].set_title(f'{phantom_name} - SSIM Dağılımı', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)

    plt.tight_layout()

    # Kaydet
    save_path = os.path.join(output_dir, f"{phantom_name}_metrics_distribution.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"  💾 Metrik dağılımı kaydedildi: {os.path.basename(save_path)}")
    plt.close()

def plot_slice_metrics(psnr_scores, ssim_scores, phantom_name, output_dir):
    """Slice bazında metrik değişimini gösterir"""
    fig, axes = plt.subplots(2, 1, figsize=(14, 8))

    slice_numbers = np.arange(len(psnr_scores))

    # PSNR over slices
    axes[0].plot(slice_numbers, psnr_scores, color='steelblue', linewidth=1.5, alpha=0.7)
    axes[0].axhline(np.mean(psnr_scores), color='red', linestyle='--', linewidth=2,
                    label=f'Ortalama: {np.mean(psnr_scores):.2f} dB')
    axes[0].fill_between(slice_numbers, psnr_scores, alpha=0.3, color='steelblue')
    axes[0].set_xlabel('Slice Numarası', fontsize=12)
    axes[0].set_ylabel('PSNR (dB)', fontsize=12)
    axes[0].set_title(f'{phantom_name} - Slice Bazında PSNR', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(alpha=0.3)

    # SSIM over slices
    axes[1].plot(slice_numbers, ssim_scores, color='forestgreen', linewidth=1.5, alpha=0.7)
    axes[1].axhline(np.mean(ssim_scores), color='red', linestyle='--', linewidth=2,
                    label=f'Ortalama: {np.mean(ssim_scores):.4f}')
    axes[1].fill_between(slice_numbers, ssim_scores, alpha=0.3, color='forestgreen')
    axes[1].set_xlabel('Slice Numarası', fontsize=12)
    axes[1].set_ylabel('SSIM', fontsize=12)
    axes[1].set_title(f'{phantom_name} - Slice Bazında SSIM', fontsize=14, fontweight='bold')
    axes[1].legend()
    axes[1].grid(alpha=0.3)

    plt.tight_layout()

    # Kaydet
    save_path = os.path.join(output_dir, f"{phantom_name}_slice_metrics.png")
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"  💾 Slice metrikleri kaydedildi: {os.path.basename(save_path)}")
    plt.close()

# ==================== MODEL YÜKLEME ====================

print("\n" + "=" * 80)
print("🔧 MODEL HAZIRLANIYOR")
print("=" * 80)

# Generator objesi kontrolü
if 'generator' not in dir():
    print("\n⚠️  UYARI: 'generator' objesi bulunamadı!")
    print("\n" + "="*80)
    print("📝 GENERATOR MODELİNİ TANIMLAYIN")
    print("="*80)
    print("\nLütfen test kodunu çalıştırmadan ÖNCE aşağıdaki kodu çalıştırın:\n")
    print("```python")
    print("# Seçenek 1: Eğer build_generator fonksiyonunuz varsa")
    print("generator = build_generator()")
    print("")
    print("# Seçenek 2: Eğer model tanımı başka bir yerdeyse")
    print("from your_model_file import build_generator")
    print("generator = build_generator()")
    print("")
    print("# Seçenek 3: Manuel olarak tanımlayın")
    print("# generator = tf.keras.models.Sequential([...])")
    print("```")
    print("\n" + "="*80)
    print("❌ Test durduruluyor. Önce generator'ı tanımlayın ve bu kodu tekrar çalıştırın.")
    print("="*80)
else:
    print("✅ Generator objesi bellekte bulundu.")
    print(f"   Model tipi: {type(generator)}")

    # Model ağırlıklarını yükle
    generator = load_latest_generator(checkpoint_dir, generator)

    if generator is None:
        print("\n❌ Generator modeli yüklenemedi. Test durduruluyor.")
        print("\n💡 Olası nedenler:")
        print("   1. Checkpoint dosyası bulunamadı")
        print("   2. Model mimarisi checkpoint ile uyumsuz")
        print("   3. Dosya yolu hatalı")
        print(f"\n   Kontrol edin: {checkpoint_dir}")
        exit()

# ==================== PHANTOM TESTLERİ ====================

print("\n" + "=" * 80)
print("🧪 PHANTOM TESTLERİ BAŞLIYOR")
print("=" * 80)

os.makedirs(output_dir, exist_ok=True)

all_results = {}

for phantom_name in phantoms:
    print(f"\n{'='*80}")
    print(f"🔬 Phantom: {phantom_name}")
    print(f"{'='*80}")

    phantom_dir = os.path.join(phantomx_processed_dir, phantom_name)

    # NPY dosyalarını yükle
    input_path = os.path.join(phantom_dir, "input_ldct.npy")
    target_path = os.path.join(phantom_dir, "target_ndct.npy")

    if not os.path.exists(input_path) or not os.path.exists(target_path):
        print(f"❌ NPY dosyaları bulunamadı: {phantom_dir}")
        print(f"   Kontrol edilen dosyalar:")
        print(f"   - {input_path}")
        print(f"   - {target_path}")
        continue

    print(f"📂 Veri yükleniyor...")
    input_volume = np.load(input_path)
    target_volume = np.load(target_path)

    print(f"  ✓ Input shape: {input_volume.shape}")
    print(f"  ✓ Target shape: {target_volume.shape}")
    print(f"  ✓ Input range: [{input_volume.min():.3f}, {input_volume.max():.3f}]")
    print(f"  ✓ Target range: [{target_volume.min():.3f}, {target_volume.max():.3f}]")

    # Model input boyutunu kontrol et
    model_input_shape = generator.input_shape  # (None, H, W, C)
    expected_h = model_input_shape[1]
    expected_w = model_input_shape[2]

    actual_h = input_volume.shape[1]
    actual_w = input_volume.shape[2]

    print(f"\n  🔍 Boyut kontrolü:")
    print(f"     Model beklenen boyut: {expected_h}x{expected_w}")
    print(f"     Veri boyutu: {actual_h}x{actual_w}")

    # Eğer boyutlar uyuşmuyorsa resize et
    needs_resize = (actual_h != expected_h) or (actual_w != expected_w)
    original_input = None
    original_target = None

    if needs_resize:
        print(f"\n  🔄 Görüntüler resize ediliyor: {actual_h}x{actual_w} → {expected_h}x{expected_w}")

        from tensorflow.keras.preprocessing.image import array_to_img, img_to_array
        import cv2

        # Input volume'u resize et
        input_resized = np.zeros((input_volume.shape[0], expected_h, expected_w), dtype=np.float32)
        for i in range(input_volume.shape[0]):
            input_resized[i] = cv2.resize(input_volume[i], (expected_w, expected_h),
                                         interpolation=cv2.INTER_LINEAR)

        # Target volume'u resize et
        target_resized = np.zeros((target_volume.shape[0], expected_h, expected_w), dtype=np.float32)
        for i in range(target_volume.shape[0]):
            target_resized[i] = cv2.resize(target_volume[i], (expected_w, expected_h),
                                          interpolation=cv2.INTER_LINEAR)

        # Orijinal boyutları sakla (sonra tekrar büyütmek için)
        original_input = input_volume
        original_target = target_volume

        input_volume = input_resized
        target_volume = target_resized

        print(f"     ✓ Resize tamamlandı: {input_volume.shape}")

    # Model için tahmin yap
    num_slices = input_volume.shape[0]
    predicted_volume = np.zeros_like(input_volume)

    print(f"\n🔮 Model tahminleri yapılıyor ({num_slices} slice)...")

    # Progress bar için
    from tqdm import tqdm

    for i in tqdm(range(num_slices), desc=f"  {phantom_name}"):
        # Slice'ı al ve reshape et: (1, H, W, 1)
        input_slice = input_volume[i:i+1, :, :]
        input_slice = np.expand_dims(input_slice, axis=-1)  # Channel dimension ekle

        # Tahmin yap
        predicted_slice = generator.predict(input_slice, verbose=0)

        # Geri çevir: (H, W)
        predicted_volume[i] = predicted_slice[0, :, :, 0]

    print(f"  ✅ Tüm tahminler tamamlandı!")

    # Eğer resize yaptıysak, tahminleri orijinal boyuta geri getir
    if needs_resize:
        print(f"\n  🔄 Tahminler orijinal boyuta geri getiriliyor: {expected_h}x{expected_w} → {actual_h}x{actual_w}")

        predicted_original_size = np.zeros_like(original_input, dtype=np.float32)
        for i in range(predicted_volume.shape[0]):
            predicted_original_size[i] = cv2.resize(predicted_volume[i], (actual_w, actual_h),
                                                    interpolation=cv2.INTER_LINEAR)

        # Metrikleri hesaplamak için orijinal boyutları kullan
        input_for_metrics = original_input
        target_for_metrics = original_target
        predicted_for_metrics = predicted_original_size

        # Görselleştirme için resize edilmiş versiyonları kullan (daha hızlı)
        input_for_viz = input_volume
        target_for_viz = target_volume
        predicted_for_viz = predicted_volume

        print(f"     ✓ Geri getirme tamamlandı: {predicted_original_size.shape}")
    else:
        input_for_metrics = input_volume
        target_for_metrics = target_volume
        predicted_for_metrics = predicted_volume

        input_for_viz = input_volume
        target_for_viz = target_volume
        predicted_for_viz = predicted_volume

    # Metrikleri hesapla
    print(f"\n📊 Metrikler hesaplanıyor...")
    psnr_scores, ssim_scores = calculate_metrics_3d(target_for_metrics, predicted_for_metrics)

    # İstatistikleri kaydet
    results = {
        'phantom': phantom_name,
        'num_slices': num_slices,
        'psnr_mean': np.mean(psnr_scores),
        'psnr_std': np.std(psnr_scores),
        'psnr_min': np.min(psnr_scores),
        'psnr_max': np.max(psnr_scores),
        'psnr_median': np.median(psnr_scores),
        'ssim_mean': np.mean(ssim_scores),
        'ssim_std': np.std(ssim_scores),
        'ssim_min': np.min(ssim_scores),
        'ssim_max': np.max(ssim_scores),
        'ssim_median': np.median(ssim_scores)
    }

    all_results[phantom_name] = results

    # Sonuçları yazdır
    print(f"\n{'─'*80}")
    print(f"  📈 SONUÇLAR - {phantom_name}")
    print(f"{'─'*80}")
    print(f"  Toplam Slice: {num_slices}")
    print(f"\n  PSNR:")
    print(f"    Ortalama: {results['psnr_mean']:.2f} dB (±{results['psnr_std']:.2f})")
    print(f"    Medyan:   {results['psnr_median']:.2f} dB")
    print(f"    Min/Max:  {results['psnr_min']:.2f} / {results['psnr_max']:.2f} dB")
    print(f"\n  SSIM:")
    print(f"    Ortalama: {results['ssim_mean']:.4f} (±{results['ssim_std']:.4f})")
    print(f"    Medyan:   {results['ssim_median']:.4f}")
    print(f"    Min/Max:  {results['ssim_min']:.4f} / {results['ssim_max']:.4f}")
    print(f"{'─'*80}\n")

    # Görselleştirmeler
    print(f"🎨 Görselleştirmeler oluşturuluyor...")
    visualize_results(input_for_viz, target_for_viz, predicted_for_viz,
                     phantom_name, output_dir, num_slices=5)
    plot_metrics_distribution(psnr_scores, ssim_scores, phantom_name, output_dir)
    plot_slice_metrics(psnr_scores, ssim_scores, phantom_name, output_dir)

    # Tahminleri kaydet (orijinal boyutta)
    pred_save_path = os.path.join(output_dir, f"{phantom_name}_predicted.npy")
    np.save(pred_save_path, predicted_for_metrics)
    print(f"  💾 Tahminler kaydedildi: {os.path.basename(pred_save_path)}\n")

# ==================== GENEL RAPOR ====================

print("\n" + "=" * 80)
print("📋 GENEL DEĞERLENDIRME RAPORU")
print("=" * 80)

if len(all_results) == 0:
    print("\n❌ Hiçbir phantom için test yapılamadı!")
else:
    # Her phantom için özet
    for phantom_name, results in all_results.items():
        print(f"\n🔬 {phantom_name}:")
        print(f"  📊 PSNR: {results['psnr_mean']:.2f} ± {results['psnr_std']:.2f} dB")
        print(f"  📊 SSIM: {results['ssim_mean']:.4f} ± {results['ssim_std']:.4f}")

    # Ortalama sonuçlar (tüm phantomlar)
    if len(all_results) > 1:
        avg_psnr = np.mean([r['psnr_mean'] for r in all_results.values()])
        avg_ssim = np.mean([r['ssim_mean'] for r in all_results.values()])

        print(f"\n{'─'*80}")
        print(f"📊 TÜM PHANTOM ORTALAMASI:")
        print(f"  PSNR: {avg_psnr:.2f} dB")
        print(f"  SSIM: {avg_ssim:.4f}")
        print(f"{'─'*80}")

    # Dosyaları listele
    print(f"\n📁 ÇIKTI DOSYALARI ({output_dir}):")
    for phantom_name in all_results.keys():
        print(f"\n  {phantom_name}/")
        print(f"    ├── {phantom_name}_comparison.png          (Görsel karşılaştırma)")
        print(f"    ├── {phantom_name}_metrics_distribution.png (Metrik dağılımları)")
        print(f"    ├── {phantom_name}_slice_metrics.png       (Slice bazında trend)")
        print(f"    └── {phantom_name}_predicted.npy           (Tahmin edilen volume)")

print("\n" + "=" * 80)
print("✅ TEST TAMAMLANDI!")
print("=" * 80)

PhantomX Model Test ve Değerlendirme
📂 Processed data: /content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Output
🧠 Model checkpoints: /content/drive/MyDrive/Dersler/Tasarım Dersi/Projects/model_checkpoints
📊 Results output: /content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Results2

🔧 MODEL HAZIRLANIYOR
✅ Generator objesi bellekte bulundu.
   Model tipi: <class 'keras.src.models.functional.Functional'>

📦 Model bulundu ve yükleniyor...
   Dosya: G_epoch_50.h5
   Epoch: 50
   Toplam 10 checkpoint bulundu
✅ Model başarıyla yüklendi!

🧪 PHANTOM TESTLERİ BAŞLIYOR

🔬 Phantom: D55-01
📂 Veri yükleniyor...
  ✓ Input shape: (376, 512, 512)
  ✓ Target shape: (376, 512, 512)
  ✓ Input range: [0.024, 1.000]
  ✓ Target range: [0.024, 1.000]

  🔍 Boyut kontrolü:
     Model beklenen boyut: 256x256
     Veri boyutu: 512x512

  🔄 Görüntüler resize ediliyor: 512x512 → 256x256
     ✓ Resize tamamlandı: (376, 256, 256)

🔮 Model tahminleri yapılıyor (376 slice)...


  D55-01: 100%|██████████| 376/376 [00:42<00:00,  8.85it/s]


  ✅ Tüm tahminler tamamlandı!

  🔄 Tahminler orijinal boyuta geri getiriliyor: 256x256 → 512x512
     ✓ Geri getirme tamamlandı: (376, 512, 512)

📊 Metrikler hesaplanıyor...

────────────────────────────────────────────────────────────────────────────────
  📈 SONUÇLAR - D55-01
────────────────────────────────────────────────────────────────────────────────
  Toplam Slice: 376

  PSNR:
    Ortalama: 36.76 dB (±1.20)
    Medyan:   37.09 dB
    Min/Max:  30.59 / 37.49 dB

  SSIM:
    Ortalama: 0.9682 (±0.0258)
    Medyan:   0.9744
    Min/Max:  0.8157 / 0.9787
────────────────────────────────────────────────────────────────────────────────

🎨 Görselleştirmeler oluşturuluyor...
  💾 Görselleştirme kaydedildi: D55-01_comparison.png
  💾 Metrik dağılımı kaydedildi: D55-01_metrics_distribution.png
  💾 Slice metrikleri kaydedildi: D55-01_slice_metrics.png
  💾 Tahminler kaydedildi: D55-01_predicted.npy


🔬 Phantom: D55-02
📂 Veri yükleniyor...
  ✓ Input shape: (363, 512, 512)
  ✓ Target shape: (36

  D55-02: 100%|██████████| 363/363 [00:32<00:00, 11.02it/s]


  ✅ Tüm tahminler tamamlandı!

  🔄 Tahminler orijinal boyuta geri getiriliyor: 256x256 → 512x512
     ✓ Geri getirme tamamlandı: (363, 512, 512)

📊 Metrikler hesaplanıyor...

────────────────────────────────────────────────────────────────────────────────
  📈 SONUÇLAR - D55-02
────────────────────────────────────────────────────────────────────────────────
  Toplam Slice: 363

  PSNR:
    Ortalama: 36.79 dB (±1.38)
    Medyan:   37.15 dB
    Min/Max:  28.26 / 38.78 dB

  SSIM:
    Ortalama: 0.9688 (±0.0258)
    Medyan:   0.9732
    Min/Max:  0.7700 / 0.9796
────────────────────────────────────────────────────────────────────────────────

🎨 Görselleştirmeler oluşturuluyor...
  💾 Görselleştirme kaydedildi: D55-02_comparison.png
  💾 Metrik dağılımı kaydedildi: D55-02_metrics_distribution.png
  💾 Slice metrikleri kaydedildi: D55-02_slice_metrics.png
  💾 Tahminler kaydedildi: D55-02_predicted.npy


📋 GENEL DEĞERLENDIRME RAPORU

🔬 D55-01:
  📊 PSNR: 36.76 ± 1.20 dB
  📊 SSIM: 0.9682 ± 0.0258



# Error Handling

In [ ]:
import pydicom
import os
import glob

# Yollarınızı buraya tanımlayın
mayo_ornek = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Hata ayıklama/1-0001.dcm" # Mayo'dan rastgele bir dosya
phantom_ornek = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/phantomx_abdomen_pelvis_dataset/D55-01/40/FBP_FC08_40_152622.890/IM-0025-0333.dcm" # PhantomX'ten rastgele bir dosya (veya NPY öncesi hali)

def check_dicom_header(path, name):
    if not os.path.exists(path): return
    ds = pydicom.dcmread(path)
    print(f"--- {name} ---")
    print(f"Intercept: {ds.RescaleIntercept if 'RescaleIntercept' in ds else 'YOK'}")
    print(f"Slope:     {ds.RescaleSlope if 'RescaleSlope' in ds else 'YOK'}")
    raw_min = ds.pixel_array.min()
    raw_max = ds.pixel_array.max()
    print(f"Raw Min/Max: {raw_min} / {raw_max}")

    # Gerçek HU değerlerini hesapla
    slope = ds.RescaleSlope if 'RescaleSlope' in ds else 1
    intercept = ds.RescaleIntercept if 'RescaleIntercept' in ds else 0
    hu_min = raw_min * slope + intercept
    hu_max = raw_max * slope + intercept
    print(f"Gerçek HU Min/Max: {hu_min} / {hu_max}\n")

check_dicom_header(mayo_ornek, "MAYO CLINIC")
check_dicom_header(phantom_ornek, "PHANTOMX")

In [ ]:
import pydicom
import os
from google.colab import drive

# 1. Google Drive'ı Bağla (Eğer bağlı değilse pencere açılır)
if not os.path.exists('/content/drive'):
    print("Google Drive bağlanıyor...")
    drive.mount('/content/drive')
else:
    print("✅ Google Drive zaten bağlı.")

# 2. Dosya Yolları (Sizin verdiğiniz yollar)
mayo_ornek = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/Hata ayıklama/1-0001.dcm"
phantom_ornek = r"/content/drive/MyDrive/Dersler/Tasarım Dersi/Phantomx/archive/phantomx_abdomen_pelvis_dataset/D55-01/40/FBP_FC08_40_152622.890/IM-0025-0333.dcm"

def check_dicom_header(path, name):
    print(f"\n🔍 {name} aranıyor...")

    # --- KONTROL KISMI ---
    if not os.path.exists(path):
        print(f"❌ HATA: Dosya bulunamadı!")
        print(f"   Aranan Yol: {path}")

        # Klasör var mı diye bakalım (Dosya adı mı yanlış?)
        folder = os.path.dirname(path)
        if os.path.exists(folder):
            print(f"   ✅ Klasör mevcut: {folder}")
            print(f"   📂 Klasör içindeki ilk 3 dosya:")
            files = os.listdir(folder)
            for f in files[:3]: print(f"      - {f}")
        else:
            print(f"   ❌ Klasör de bulunamadı. Yol tamamen yanlış olabilir.")
        return
    # ---------------------

    ds = pydicom.dcmread(path)
    print(f"✅ Dosya bulundu. Analiz ediliyor...")
    print(f"-"*30)

    # Header Bilgileri
    intercept = ds.RescaleIntercept if 'RescaleIntercept' in ds else 0
    slope = ds.RescaleSlope if 'RescaleSlope' in ds else 1

    print(f"Rescale Intercept: {intercept}")
    print(f"Rescale Slope:     {slope}")

    raw_min = ds.pixel_array.min()
    raw_max = ds.pixel_array.max()
    print(f"Raw Pixel Min/Max: {raw_min} / {raw_max}")

    # Gerçek HU
    hu_min = raw_min * slope + intercept
    hu_max = raw_max * slope + intercept
    print(f"Gerçek HU Min/Max: {hu_min:.1f} / {hu_max:.1f}")

    # YORUM
    if intercept == -1024:
        print("💡 YORUM: Bu veri 'Offset' kullanıyor. (Tipik Mayo/Siemens/GE)")
        print("   Eğitimde bu -1024'ü hesaba katmadıysanız model hatalı öğrendi.")
    elif intercept == 0:
        print("💡 YORUM: Bu veri zaten HU'ya yakın veya farklı formatta.")

# Analizi Başlat
check_dicom_header(mayo_ornek, "MAYO CLINIC")
check_dicom_header(phantom_ornek, "PHANTOMX")

Google Drive bağlanıyor...
Mounted at /content/drive

🔍 MAYO CLINIC aranıyor...
✅ Dosya bulundu. Analiz ediliyor...
------------------------------
Rescale Intercept: -0.9999866485596
Rescale Slope:     0.00032043788815
Raw Pixel Min/Max: 202 / 61026
Gerçek HU Min/Max: -0.9 / 18.6

🔍 PHANTOMX aranıyor...
✅ Dosya bulundu. Analiz ediliyor...
------------------------------
Rescale Intercept: 0
Rescale Slope:     1
Raw Pixel Min/Max: -2048 / 1521
Gerçek HU Min/Max: -2048.0 / 1521.0
💡 YORUM: Bu veri zaten HU'ya yakın veya farklı formatta.
